<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Instructions" data-toc-modified-id="Instructions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Instructions</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np

from scipy.spatial.distance import euclidean

In [2]:
DF_COOR = pd.DataFrame(                                 #Make this a global variable so that it doesn't need to be passed into 
      [['A01', -1120.0, 0.0, 85.2, np.nan, np.nan],     #distance function every time
       ['A02', -1085.0, -200.0, 84.6, np.nan, np.nan],
       ['A03', -1085.0, 200.0, 84.6, np.nan, np.nan],
       ['A04', -934.0, 0.0, 83.2, np.nan, np.nan],
       ['A05', -995.0, -586.0, 85.3, np.nan, np.nan],
       ['A06', -1055.0, -372.9, 77.5, np.nan, np.nan],
       ['A07', -893.0, -506.0, 91.4, np.nan, np.nan],
       ['A08', -972.0, -372.9, 90.9, np.nan, np.nan],
       ['A09', -780.0, -506.0, 123.4, np.nan, np.nan],
       ['A10', -885.0, -372.9, 128.4, np.nan, np.nan],
       ['A11', -615.0, -615.0, 154.7, np.nan, np.nan],
       ['A12', -995.0, 586.0, 85.3, np.nan, np.nan],
       ['A13', -1055.0, 372.9, 77.5, np.nan, np.nan],
       ['A14', -893.0, 506.0, 91.4, np.nan, np.nan],
       ['A15', -972.0, 372.9, 90.9, np.nan, np.nan],
       ['A16', -780.0, 506.0, 123.4, np.nan, np.nan],
       ['A17', -885.0, 372.9, 128.4, np.nan, np.nan],
       ['A18', -615.0, 615.0, 154.7, np.nan, np.nan],
       ['A19', -880.4, -800.0, 78.0, np.nan, np.nan],
       ['A20', -740.0, -850.0, 105.7, np.nan, np.nan],
       ['A21', -625.0, -800.0, 76.5, np.nan, np.nan],
       ['A22', -474.8, -843.7, 92.7, np.nan, np.nan],
       ['A23', -880.4, 800.0, 78.0, np.nan, np.nan],
       ['A24', -740.0, 850.0, 105.7, np.nan, np.nan],
       ['A25', -625.0, 800.0, 76.5, np.nan, np.nan],
       ['A26', -474.8, 843.7, 92.7, np.nan, np.nan],
       ['A27', -760.0, -730.0, 123.3, np.nan, np.nan],
       ['A28', -490.7, -805.0, 115.2, np.nan, np.nan],
       ['A29', -461.2, -828.8, 145.0, np.nan, np.nan],
       ['A30', -760.0, 730.0, 123.3, np.nan, np.nan],
       ['A31', -490.7, 805.0, 115.2, np.nan, np.nan],
       ['A32', -461.2, 828.8, 145.0, np.nan, np.nan]], 
                       
       columns=['NAME', 'X', 'Y', 'Z', 'nearest', '2nd_nearest'])

In [3]:
def distance(idx1, idx2):
    """Calculates a distance between two points. It takes two 
    df indexes and returns the euclidean distance """
    
    r = euclidean((DF_COOR['X'][idx1], DF_COOR['Y'][idx1], DF_COOR['Z'][idx1]),
                   (DF_COOR['X'][idx2], DF_COOR['Y'][idx2], DF_COOR['Z'][idx2]))
    return r

distance(31, 30)

48.215453954100624

In [4]:
#init distances to inf
DF_COOR["nearest_dist"], DF_COOR["2nd_nearest_dist"] = np.inf, np.inf

DF_COOR["nearest"], DF_COOR["2nd_nearest"] = DF_COOR["nearest"].astype(str), DF_COOR["2nd_nearest"].astype(str)
DF_COOR.head()

,NAME,X,Y,Z,nearest,2nd_nearest,nearest_dist,2nd_nearest_dist
0,A01,-1120.0,0.0,85.2,nan,nan,inf,inf
1,A02,-1085.0,-200.0,84.6,nan,nan,inf,inf
2,A03,-1085.0,200.0,84.6,nan,nan,inf,inf
3,A04,-934.0,0.0,83.2,nan,nan,inf,inf
4,A05,-995.0,-586.0,85.3,nan,nan,inf,inf


In [5]:
def check_and_update_dists(r, idx_x, idx_y):
    """Helper function. Checks if calculated distance r is 
    current nearest or second nearest point and updates 
    DF_COOR if so."""
    if r < DF_COOR["nearest_dist"][idx_x]:
        DF_COOR.at[idx_x, "nearest_dist"] = r
        DF_COOR.at[idx_x, "nearest"] =  DF_COOR["NAME"][idx_y]
    elif r < DF_COOR["2nd_nearest_dist"][idx_x]:
        DF_COOR.at[idx_x, "2nd_nearest_dist"] = r
        DF_COOR.at[idx_x, "2nd_nearest"] =  DF_COOR["NAME"][idx_y]

In [6]:
#I am assuming that distance() is most expensive
#part of this routine so I have chosen a method
#that attempts to minimize the number of calls
#to this function. This is minimized by 
#not duplicating the calls and keeping a record 
#of shortest distances

for idx_x in DF_COOR.index:
    for idx_y in range(idx_x + 1, len(DF_COOR.index)): #check all later points
        r = distance(idx_x, idx_y)
        check_and_update_dists(r, idx_x, idx_y)
        #now update the second point with new dist 
        check_and_update_dists(r, idx_y, idx_x)
DF_COOR

,NAME,X,Y,Z,nearest,2nd_nearest,nearest_dist,2nd_nearest_dist
0,A01,-1120.0,0.0,85.2,A04,A03,186.010752,203.040292
1,A02,-1085.0,-200.0,84.6,A06,A08,175.626934,206.647284
2,A03,-1085.0,200.0,84.6,A13,A15,175.626934,206.647284
3,A04,-934.0,0.0,83.2,A01,A02,186.010752,250.605187
4,A05,-995.0,-586.0,85.3,A07,A08,129.773688,214.410751
5,A06,-1055.0,-372.9,77.5,A08,A10,84.074729,177.456502
6,A07,-893.0,-506.0,91.4,A09,A10,117.443603,138.378503
7,A08,-972.0,-372.9,90.9,A06,A10,84.074729,94.737796
8,A09,-780.0,-506.0,123.4,A07,A10,117.443603,169.604275
9,A10,-885.0,-372.9,128.4,A08,A09,94.737796,169.604275


In [8]:
DF_COOR.drop(columns=["nearest_dist", "2nd_nearest_dist"])

,NAME,X,Y,Z,nearest,2nd_nearest
0,A01,-1120.0,0.0,85.2,A04,A03
1,A02,-1085.0,-200.0,84.6,A06,A08
2,A03,-1085.0,200.0,84.6,A13,A15
3,A04,-934.0,0.0,83.2,A01,A02
4,A05,-995.0,-586.0,85.3,A07,A08
5,A06,-1055.0,-372.9,77.5,A08,A10
6,A07,-893.0,-506.0,91.4,A09,A10
7,A08,-972.0,-372.9,90.9,A06,A10
8,A09,-780.0,-506.0,123.4,A07,A10
9,A10,-885.0,-372.9,128.4,A08,A09
